In [6]:
import math
import scipy as sp
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import gaussian_kde

from qiskit import QuantumCircuit, transpile, assemble
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import QiskitRuntimeService , SamplerV2
from qiskit.quantum_info import Operator
from qiskit.providers.fake_provider import GenericBackendV2
from qiskit.providers.basic_provider import BasicSimulator
from qiskit.visualization import plot_histogram
from qiskit.visualization import array_to_latex
from qiskit_aer import StatevectorSimulator, AerSimulator
from qiskit_aer.noise import NoiseModel, depolarizing_error
from qiskit_ibm_runtime.fake_provider import FakeKyiv
import qiskit_algorithms
from qiskit_algorithms.optimizers import COBYLA
from qiskit.quantum_info import Statevector
import random

In [7]:
#DEFINE 8x8 matrix 

I = np.array([[1,0],[0,1]])
X = np.array([[0,1],[1,0]])
Z = np.array([[1,0],[0,-1]])
pauli_mat = {0: I, 1: Z, 2: X}

def kron3(code):
    return np.kron(np.kron(pauli_mat[code[2]], pauli_mat[code[1]]), pauli_mat[code[0]]) #ORDER IN PAULI MARTICES!!!!!
    
# A =  0.7 (IIZ)  + 0.4 (XII)  – 0.3 (ZXX)  + 0.2 (IXI)  + 0.1 (XXX)

chosen_terms = {
    (0,0,1):  0.7,    # I ⊗ I ⊗ Z
    (2,0,0):  0.4,    # X ⊗ I ⊗ I
    (1,2,2): -0.3,    # Z ⊗ X ⊗ X
    (0,2,0):  0.2,    # I ⊗ X ⊗ I
    (2,2,2):  0.1,    # X ⊗ X ⊗ X
}

A = sum(c * kron3(code) for code, c in chosen_terms.items())
array_to_latex(A)

<IPython.core.display.Latex object>

In [8]:
n = 3 #variational qubits
q = n + 1 #all qubits 1 ancilla

In [10]:
#construct b or maybe deconstruct b
b = np.array([9,5,4,6,2,8,7,5])  
b_norm = b / np.linalg.norm(b)
initialn= int(np.log2(len(b_norm)))

In [11]:
#VARIATIONAL ANSATZ

def apply_fixed_ansatz(var,qubits, param):

    for i in range(0,n):
        var.ry(param[0][i],qubits[i] )
        
    var.cz(qubits[0], qubits[1])
    var.cz(qubits[2], qubits[0])

    for  j in range (0, len(qubits)):
        var.ry(param[1][j], qubits[j])

    var.cz(qubits[1], qubits[2])
    var.cz(qubits[2], qubits[0])

    for k in range (0, len(qubits)):
        var.ry(param[2][k], qubits[k])


var = QuantumCircuit(q)
apply_fixed_ansatz(var,[1,2,3], [[1, 1, 1], [1, 1, 1], [1, 1, 1]])
#var.draw("mpl")

In [12]:
#DEFINITION Y CHECKEO DE LA CONSTRUCCION DE U 
def Uprep(b):
# Usa el Gram-Schmidt method
    b = np.asarray(b, dtype=float).reshape(8)
    b = b / np.linalg.norm(b)
    
    basis = [b]

    for i in range(8):
        v = np.zeros(8)
        v[i] = 1.0
        for u in basis:
            v -= np.dot(u, v) * u
        norm_v = np.linalg.norm(v)
        if norm_v > 1e-10:
            basis.append(v / norm_v)
        if len(basis) == 8:
            break

    U = np.column_stack(basis)
    return U

In [13]:
qubits  =[1,2,3]
shots = 1024
chosen_terms = {
    (0,0,1):  0.7,    # I ⊗ I ⊗ Z
    (2,0,0):  0.4,    # X ⊗ I ⊗ I
    (1,2,2): -0.3,    # Z ⊗ X ⊗ X
    (0,2,0):  0.2,    # I ⊗ X ⊗ I
    (2,2,2):  0.1,    # X ⊗ X ⊗ X
}  
norm_factor = 2**n  
projector_terms = {
    k: coeff / norm_factor
    for k, coeff in chosen_terms.items()
}
backend = GenericBackendV2(q)
storecounts = []
U_gate = Operator(Uprep(b)).to_instruction().control(1)
def iterationQ(chosen_terms, shots, qubits, param):
    storecounts = []
    for k in chosen_terms:
        var = QuantumCircuit(q,1)
        apply_fixed_ansatz(var, qubits, param)
        var.h(0)
        for i in range(0,n):
            if k[i] == 1:
                var.cz(0,qubits[i])
            if k[i] == 2:
                var.cx(0,qubits[i])
        var.append(U_gate, [0,1,2,3])
        var.h(0)
        var.measure(0,0)
        transpiled_circuit = transpile(var, backend) 
        job = backend.run(transpiled_circuit, shots = shots, noise_model=None)
        counts = job.result().get_counts()
        storecounts.append(counts)
    coeffs = list(projector_terms.values())
    Expec = []
    for i in range(0,len(storecounts)):
        Expec.append(coeffs[i]*(storecounts[i].get("0", 0)/shots - storecounts[i].get("1", 0)/shots))
    overlap = sum(Expec)
    return overlap

In [ ]:
def cost(param):
    return 1 - iterationQ(chosen_terms, shots, qubits, param)

def cost_flat(flat_param):
    param_matrix = flat_param.reshape((3, 3))
    cost_value = 1 - iterationQ(chosen_terms, shots, qubits, param_matrix)
    cost_history.append(cost_value)
    print(cost_value)
    return cost_value
cost_history = []
res = COBYLA(maxiter=200).minimize(fun=cost_flat, x0 = np.random.uniform(0, 2*np.pi, size=9))


plt.plot(cost_history, 'o-')
plt.xlabel("Iteración")
plt.ylabel("Coste C(θ)")
plt.title("Convergencia VQLS")
plt.show()


print("Ultima iteracion =", res.fun)
print("Mejor solapamiento² =", 1 - res.fun)
print("Parámetros óptimos =", res.x.reshape((3, 3)))
